In [1]:
import cv2
import numpy as np

# Method Definitions

In [2]:
def render(image, win_name="MyWindow"):
    cv2.imshow(win_name, image)
    cv2.waitKey(0)
    cv2.destroyWindow(win_name)

In [3]:
def compare(image1, image2):
    scale = .5
    image1_r = cv2.resize(image1, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
    image2_r = cv2.resize(image2, None, fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
    compare = np.concat((image1, image2), axis=1)
    render(compare)

In [4]:
def region_of_interest(image):
    height = image.shape[0]
    polygons = np.array([[(200, height), (1100, height), (550, 250)]])
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [5]:
def make_coordinates(image, line_parameters):
    slope, intercept = line_parameters
    y1 = image.shape[0]
    y2 = int(y1*(3/5))
    x1 = int((y1 - intercept)/slope)
    x2 = int((y2 - intercept)/slope)
    return np.array([x1, y1, x2, y2])

In [6]:
def average_slope_intercept(image, lines):
    left_fit = []
    right_fit = []
    for line in lines:
        x1, y1, x2, y2 = line.reshape(4)
        parameters = np.polyfit((x1, x2), (y1, y2), 1)
        slope = parameters[0]
        intercept = parameters[1]
        if slope < 0:
            left_fit.append((slope, intercept))
        else:
            right_fit.append((slope, intercept))
    left_fit_average = np.average(left_fit, axis=0)
    right_fit_average = np.average(right_fit, axis=0)
    left_line = make_coordinates(image, left_fit_average)
    right_line = make_coordinates(image, right_fit_average)
    return np.array([left_line, right_line])

In [7]:
def region_of_interest(image, render_mask=False):
    height = image.shape[0]
    width = image.shape[1]
    polygons = np.array([[[0, height], [width, height], [width/2, height/2]]], dtype=np.int32)
    mask = np.zeros_like(image)
    cv2.fillPoly(mask, polygons, 255)
    if render_mask: render(cv2.addWeighted(mask, 0.3, image, 1, 0), "Region of Interest Mask")
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [8]:
def detect_lanes(image, render_steps=False):
    lanes_image = np.copy(image)
    if render_steps:
        render(lanes_image, "Original")
    lanes_image = cv2.cvtColor(lanes_image, cv2.COLOR_RGB2GRAY)
    if render_steps:
        render(lanes_image, "Grayscale")
    lanes_image = cv2.GaussianBlur(lanes_image, (5, 5), 0)
    if render_steps:
        render(lanes_image, "Gaussian Blur")
    lanes_image = cv2.Canny(lanes_image, 50, 150)
    if render_steps:
        render(lanes_image, "Canny")
    lanes_image = region_of_interest(lanes_image, True)
    if render_steps:
        render(lanes_image, "Region of Interest")
    lines = cv2.HoughLinesP(lanes_image, 2, np.pi/180, 100, np.array([]), minLineLength=40, maxLineGap=5)
    lines = average_slope_intercept(lanes_image, lines)
    lines_image = np.zeros_like(image)
    for line in lines:
        x1, y1, x2, y2 = line.flatten()
        cv2.line(lines_image, (x1, y1), (x2, y2), (255, 0, 255), 10)
    return cv2.addWeighted(image, 0.6, lines_image, 1, 1)

# Testing

In [9]:
image1 = cv2.imread("../Data/test1.jpg")

In [10]:
render(detect_lanes(image1, render_steps=True))

In [11]:
image2 = cv2.imread("../Data/test2.webp")

In [12]:
render(image2)

error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:1261: error: (-27:Null pointer) NULL window: 'MyWindow' in function 'cvDestroyWindow'


In [ ]:
render(detect_lanes(image2, True))